In [1]:
!pip show azureml-train-automl

Name: azureml-train-automl
Version: 1.31.0
Summary: automl
Home-page: https://docs.microsoft.com/python/api/overview/azure/ml/?view=azure-ml-py
Author: Microsoft Corp
Author-email: None
License: https://aka.ms/azureml-sdk-license
Location: /anaconda/envs/azureml_py36/lib/python3.6/site-packages
Requires: azureml-train-automl-client, azureml-automl-runtime, azureml-automl-core, azureml-dataset-runtime, azureml-train-automl-runtime
Required-by: azureml-automl-dnn-nlp


In [2]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.31.0 to work with jp-cpet-eastus-research-ml


In [3]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()
# Split the dataset into training and validation subsets
cpet_ds = ws.datasets.get("cpet-pulmonary-dataset-selected")
train_ds, test_ds = cpet_ds.random_split(percentage=0.7, seed=123)
print("Data ready!")

Data ready!


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "jjp581"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Found existing cluster, use it.


In [5]:
import azureml.train.automl.utilities as automl_utils

for metric in automl_utils.get_primary_metrics('classification'):
    print(metric)

AUC_weighted
precision_score_weighted
accuracy
norm_macro_recall
average_precision_score_weighted


In [6]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(name='Automated ML Experiment Pulmonary',
                             task='classification',
                             compute_target=training_cluster,
                             training_data = train_ds,
                             validation_data = test_ds,
                             label_column_name='PulmonaryLim',
                             iterations=50,
                             primary_metric = 'norm_macro_recall',
                             max_concurrent_iterations=5,
                             featurization='auto',
                             enable_voting_ensemble = False,
                             enable_stack_ensemble = False
                             )

print("Ready for Auto ML run.")

Ready for Auto ML run.


In [7]:
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

print('Submitting Auto ML experiment...')
automl_experiment = Experiment(ws, 'cpet-automl-pulmonary-full')
automl_run = automl_experiment.submit(automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

Submitting Auto ML experiment...
Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
cpet-automl-pulmonary-full,AutoML_5489d407-f8a9-4a04-92a9-505248c84d98,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
cpet-automl-pulmonary-full,AutoML_5489d407-f8a9-4a04-92a9-505248c84d98,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS

{'runId': 'AutoML_5489d407-f8a9-4a04-92a9-505248c84d98',
 'target': 'jjp581',
 'status': 'Completed',
 'startTimeUtc': '2021-07-05T13:22:25.264891Z',
 'endTimeUtc': '2021-07-05T13:43:08.763323Z',
 'properties': {'num_iterations': '30',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'norm_macro_recall',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'jjp581',
  'AMLSettingsJsonString': '{"path":null,"name":"Automated ML Experiment Pulmonary","subscription_id":"2ea3a7ac-8926-4694-9e2c-242e69548c78","resource_group":"jp-cpet-research","workspace_name":"jp-cpet-eastus-research-ml","region":"eastus","compute_target":"jjp581","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":30,"primary_metric":"norm_macro_